In [1]:
from pathlib import Path
import random
from ultralytics.models import YOLO
import torch
import yaml
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

random.seed(42)

In [2]:
from utils import DisplayPath
Path = DisplayPath

## Step 2: Configure Paths & Hyperparameters

In [3]:
# Dataset path (created by e2e_data_prep.ipynb)
YOLO_DATASET = Path("datasets/ready/full_dataset")
RUNS_DIR = Path("runs/segment")

# Verify dataset exists
if not YOLO_DATASET.exists():
    raise FileNotFoundError(f"Dataset not found at {YOLO_DATASET}. Run e2e_data_prep.ipynb first!")

print("Dataset:")
YOLO_DATASET.display()
print("  Train:")
(YOLO_DATASET / 'train').display()
print("  Val:")
(YOLO_DATASET / 'val').display()
print("  Test:")
(YOLO_DATASET / 'test').display()

Dataset:


[datasets/ready/full_dataset](datasets/ready/full_dataset)

  Train:


[datasets/ready/full_dataset/train](datasets/ready/full_dataset/train)

  Val:


[datasets/ready/full_dataset/val](datasets/ready/full_dataset/val)

  Test:


[datasets/ready/full_dataset/test](datasets/ready/full_dataset/test)

In [4]:
EPOCHS = 50
BATCH_SIZE = 16
IMG_SIZE = 640
model_type = "yolo11n-seg.pt"
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(f"Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA: {torch.version.cuda}")

Device: cuda:0
GPU: NVIDIA GeForce RTX 3080 Laptop GPU
CUDA: 12.8


In [5]:
AUG_CONFIG = {
    'hsv_h': 0.015,  # Hue augmentation
    'hsv_s': 0.7,    # Saturation
    'hsv_v': 0.4,    # Value
    'degrees': 10.0,  # Rotation
    'translate': 0.1, # Translation
    'scale': 0.5,     # Scaling
    'shear': 0.0,     # Shearing
    'perspective': 0.0, # Perspective
    'flipud': 0.0,    # Vertical flip
    'fliplr': 0.5,    # Horizontal flip
    'mosaic': 1.0,    # Mosaic augmentation
    'mixup': 0.0,     # Mixup augmentation
}

## Step 3: Verify Dataset Structure

Dataset is already prepared by e2e_data_prep.ipynb

In [6]:
# Verify dataset structure
print("="*60)
print("DATASET VERIFICATION")
print("="*60)

splits = ['train', 'val', 'test']
stats = {}

for split in splits:
    img_dir = YOLO_DATASET / split / "images"
    lbl_dir = YOLO_DATASET / split / "labels"
    
    if img_dir.exists() and lbl_dir.exists():
        num_images = len(list(img_dir.glob("*")))
        num_labels = len(list(lbl_dir.glob("*.txt")))
        stats[split] = {'images': num_images, 'labels': num_labels}
        print(f"{split.upper():5s}: {num_images:4d} images, {num_labels:4d} labels")
    else:
        stats[split] = {'images': 0, 'labels': 0}
        print(f"{split.upper():5s}: Missing!")

total_images = sum(s['images'] for s in stats.values())
total_labels = sum(s['labels'] for s in stats.values())

print(f"{'TOTAL':5s}: {total_images:4d} images, {total_labels:4d} labels")
print("="*60)

if total_images == 0:
    raise RuntimeError("No dataset found! Run e2e_data_prep.ipynb to create the dataset.")

DATASET VERIFICATION
TRAIN:  998 images,  998 labels
VAL  :   47 images,   47 labels
TEST :  209 images,  209 labels
TOTAL: 1254 images, 1254 labels


## Step 4: Create YOLO Configuration File

In [7]:
classes = {
    'red ball': 0,
    'human': 1,
    'trashcan': 2
}

config = {
    'path': str(YOLO_DATASET.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': len(classes),
    'names': list(classes.keys())
}

config_path = YOLO_DATASET / 'data.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"✓ Configuration saved: {config_path}")
print("Dataset structure:")
YOLO_DATASET.display()
print("  Train:")
(YOLO_DATASET / 'train').display()
print("  Val:")
(YOLO_DATASET / 'val').display()
print("  Test:")
(YOLO_DATASET / 'test').display()

✓ Configuration saved: datasets/ready/full_dataset/data.yaml
Dataset structure:


[datasets/ready/full_dataset](datasets/ready/full_dataset)

  Train:


[datasets/ready/full_dataset/train](datasets/ready/full_dataset/train)

  Val:


[datasets/ready/full_dataset/val](datasets/ready/full_dataset/val)

  Test:


[datasets/ready/full_dataset/test](datasets/ready/full_dataset/test)

## Step 4.5: Find Trashcan Images for Monitoring

Identify validation images containing trashcans (class 2) to monitor training progress

In [8]:
def find_trashcan_images(val_labels_dir, val_images_dir, max_images=8):
    """Find validation images that contain trashcan annotations (class 2)"""
    trashcan_images = []
    
    # Scan all label files
    label_files = list(val_labels_dir.glob("*.txt"))
    print(f"Scanning {len(label_files)} label files for trashcans...")
    
    for label_file in label_files:
        try:
            with open(label_file, 'r') as f:
                lines = f.readlines()
                
            # Check if any line starts with "2 " (trashcan class)
            has_trashcan = any(line.strip().startswith('2 ') for line in lines)
            
            if has_trashcan:
                # Find corresponding image
                img_name = label_file.stem
                
                # Try different image extensions
                for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
                    img_path = val_images_dir / f"{img_name}{ext}"
                    if img_path.exists():
                        trashcan_images.append(str(img_path))
                        print(f"  ✓ Found trashcan in: {img_path.name}")
                        break
                
                if len(trashcan_images) >= max_images:
                    break
        except Exception as e:
            # Skip problematic files (e.g., too large)
            continue
    
    return trashcan_images

# Find trashcan images
val_labels_dir = YOLO_DATASET / 'val' / 'labels'
val_images_dir = YOLO_DATASET / 'val' / 'images'

TRASHCAN_MONITOR_IMAGES = find_trashcan_images(val_labels_dir, val_images_dir, max_images=8)

print(f"\n{'='*60}")
print(f"Selected {len(TRASHCAN_MONITOR_IMAGES)} images for trashcan monitoring:")
for img_path in TRASHCAN_MONITOR_IMAGES:
    print(f"  - {Path(img_path).name}")
print(f"{'='*60}\n")

if len(TRASHCAN_MONITOR_IMAGES) == 0:
    print("⚠️  Warning: No trashcan images found in validation set!")

Scanning 47 label files for trashcans...


  ✓ Found trashcan in: de96513c1cdccc864e7b7e809162d06c.jpg
  ✓ Found trashcan in: 1383c4a58a26c7238fbae31ec8e4e660.jpg
  ✓ Found trashcan in: f7c36eabf5a95cf548a382f4a6b49050.jpg
  ✓ Found trashcan in: 380fca853fd511698abe9f0c24285f68.jpg
  ✓ Found trashcan in: 8dad2db8365784c4b162442bd59eb5c7.jpg
  ✓ Found trashcan in: d8c811abffbf5da563ee451592a24dcd.jpg
  ✓ Found trashcan in: 98a670c874d1cefcacd71b855d29769a.jpg
  ✓ Found trashcan in: b6fe1fc46e4ad868193c424070da1e34.jpg

Selected 8 images for trashcan monitoring:
  - de96513c1cdccc864e7b7e809162d06c.jpg
  - 1383c4a58a26c7238fbae31ec8e4e660.jpg
  - f7c36eabf5a95cf548a382f4a6b49050.jpg
  - 380fca853fd511698abe9f0c24285f68.jpg
  - 8dad2db8365784c4b162442bd59eb5c7.jpg
  - d8c811abffbf5da563ee451592a24dcd.jpg
  - 98a670c874d1cefcacd71b855d29769a.jpg
  - b6fe1fc46e4ad868193c424070da1e34.jpg



## Step 4.6: Define Trashcan Monitoring Callback

Create a callback that visualizes trashcan segmentation progress at each epoch

In [ ]:
def create_trashcan_monitor_callback(model, monitor_images, output_dir, project_name):
    """Create a callback to monitor trashcan segmentation progress during training"""
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    def on_train_epoch_end(trainer):
        """Called at the end of each training epoch"""
        if len(monitor_images) == 0:
            return
        
        epoch = trainer.epoch
        
        # Skip first few epochs to save time
        if epoch < 5:
            return
        
        # Run predictions on monitor images
        print(f"\n{'='*60}")
        print(f"Generating trashcan monitoring visualizations for epoch {epoch}...")
        print(f"{'='*60}")
        
        # Save current training state
        was_training = trainer.model.training
        
        try:
            # Put model in eval mode and disable gradients
            trainer.model.eval()
            
            # Create figure with subplots
            n_images = len(monitor_images)
            n_cols = min(3, n_images)
            n_rows = (n_images + n_cols - 1) // n_cols
            
            fig = plt.figure(figsize=(6 * n_cols, 5 * n_rows))
            gs = GridSpec(n_rows, n_cols, figure=fig, hspace=0.3, wspace=0.3)
            
            with torch.no_grad():
                for idx, img_path in enumerate(monitor_images):
                    try:
                        # Use the YOLO wrapper's predict method
                        # Create temporary YOLO instance with current weights
                        temp_model = YOLO(trainer.best if hasattr(trainer, 'best') else trainer.last)
                        results = temp_model.predict(img_path, conf=0.25, verbose=False)
                        
                        # Get the plotted image
                        result = results[0]
                        plotted_img = result.plot()
                        
                        # Convert BGR to RGB
                        plotted_img = cv2.cvtColor(plotted_img, cv2.COLOR_BGR2RGB)
                        
                        # Add to subplot
                        row = idx // n_cols
                        col = idx % n_cols
                        ax = fig.add_subplot(gs[row, col])
                        ax.imshow(plotted_img)
                        ax.axis('off')
                        
                        # Count detections by class
                        if result.masks is not None and len(result.boxes) > 0:
                            classes_detected = result.boxes.cls.cpu().numpy()
                            n_balls = int((classes_detected == 0).sum())
                            n_humans = int((classes_detected == 1).sum())
                            n_trashcans = int((classes_detected == 2).sum())
                            
                            title = f"{Path(img_path).name}\n"
                            title += f"Balls: {n_balls} | Humans: {n_humans} | Trashcans: {n_trashcans}"
                        else:
                            title = f"{Path(img_path).name}\nNo detections"
                        
                        ax.set_title(title, fontsize=10)
                        
                    except Exception as e:
                        print(f"  ⚠️  Error processing {Path(img_path).name}: {e}")
            
            # Add main title
            fig.suptitle(f"Trashcan Monitoring - Epoch {epoch} - {project_name}", 
                         fontsize=16, fontweight='bold', y=0.995)
            
            # Save figure
            output_file = output_dir / f"epoch_{epoch:03d}.jpg"
            plt.savefig(output_file, dpi=150, bbox_inches='tight')
            plt.close(fig)
            
            print(f"✓ Saved monitoring visualization:")
            output_dir.display()
            print(f"{'='*60}\n")
            
        finally:
            # Restore training mode
            if was_training:
                trainer.model.train()
    
    return on_train_epoch_end

## Step 5: Train Model with Trashcan Monitoring

Train YOLOv11 with:
- Data augmentation on train set
- Checkpoints saved for best model
- Validation after each epoch
- **Custom callback to monitor trashcan segmentation progress**

In [10]:
# Load pretrained model
model = YOLO(model_type)

In [11]:
project_name = 'ball_person_trashcan_model_v3'

In [ ]:
# Setup trashcan monitoring
monitor_output_dir = RUNS_DIR / project_name / 'trashcan_monitor'
print(f"Trashcan monitoring output: {monitor_output_dir}")

# Add callback
callback_fn = create_trashcan_monitor_callback(
    model=model,
    monitor_images=TRASHCAN_MONITOR_IMAGES,
    output_dir=monitor_output_dir,
    project_name=project_name
)

model.add_callback('on_train_epoch_end', callback_fn)
print("✓ Trashcan monitoring callback registered")

Trashcan monitoring output: runs/segment/ball_person_trashcan_model_v3/trashcan_monitor
✓ Trashcan monitoring callback registered


: 

In [ ]:
# Train model
head_idx = next((i for i, m in enumerate(model.model.model) if 'Detect' in m.__class__.__name__ or 'Segment' in m.__class__.__name__), len(model.model.model) - 1)

results = model.train(
    data=str(config_path),
    epochs=EPOCHS,
    freeze=list(range(head_idx)),
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    device=DEVICE,
    project=str(RUNS_DIR),
    name=project_name,
    exist_ok=True,
    
    # Checkpointing
    save=True,
    save_period=5,  # Save every 5 epochs
    
    # Validation
    val=True,
    
    # Data augmentation (only applied to train)
    **AUG_CONFIG,
    
    # Optimizer
    optimizer='Adam',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    
    # Loss weights
    box=7.5,
    cls=0.5,
    dfl=1.5,
    
    # Other
    patience=20,  # Early stopping
    workers=8,
    verbose=True
)

New https://pypi.org/project/ultralytics/8.3.246 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.235 🚀 Python-3.12.10 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 8192MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/ready/full_dataset/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22], half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=tra

## Step 6: Load Best Model & Evaluate

In [ ]:
best_model_path = RUNS_DIR / project_name / 'weights' / 'best.pt'
best_model_path.display()
model = YOLO(best_model_path)

## Step 7: Evaluate Results

In [ ]:
# Validation metrics
metrics = model.val()

print("\n" + "="*60)
print("VALIDATION METRICS")
print("="*60)
print(f"Box mAP50: {metrics.box.map50:.4f}")
print(f"Box mAP50-95: {metrics.box.map:.4f}")
print(f"Mask mAP50: {metrics.seg.map50:.4f}")
print(f"Mask mAP50-95: {metrics.seg.map:.4f}")

# Per-class metrics
print("\n" + "="*60)
print("PER-CLASS METRICS (Segmentation)")
print("="*60)
class_names = ['red ball', 'human', 'trashcan']
for i, class_name in enumerate(class_names):
    try:
        map50 = metrics.seg.map50_per_class[i] if hasattr(metrics.seg, 'map50_per_class') else 0
        map_val = metrics.seg.map_per_class[i] if hasattr(metrics.seg, 'map_per_class') else 0
        print(f"{class_name:12s}: mAP50={map50:.4f}, mAP50-95={map_val:.4f}")
    except:
        print(f"{class_name:12s}: metrics not available")
print("="*60)

In [ ]:
# Find best checkpoint
model_dir = RUNS_DIR / project_name
best_model = model_dir / 'weights' / 'best.pt'
last_model = model_dir / 'weights' / 'last.pt'

print(f"Best model: ")
best_model.display()
print(f"Last model: ")
last_model.display()
print(f"Results: ")
model_dir.display()
print(f"\nTrashcan monitoring visualizations: ")
(model_dir / 'trashcan_monitor').display()

## Step 8: Visualize Trashcan Progress Evolution

Create a comparison showing how trashcan segmentation improved over epochs

In [ ]:
# List all monitoring visualizations
monitor_dir = RUNS_DIR / project_name / 'trashcan_monitor'

if monitor_dir.exists():
    viz_files = sorted(monitor_dir.glob("epoch_*.jpg"))
    print(f"Found {len(viz_files)} monitoring visualizations:")
    for viz_file in viz_files:
        print(f"  - {viz_file.name}")
    
    if len(viz_files) > 0:
        print(f"\n💡 Tip: Open the images in {monitor_dir} to see how trashcan segmentation evolved!")
        print(f"   You can use an image viewer or VS Code to flip through them chronologically.")
else:
    print("No monitoring visualizations found.")

## Step 9: Test on Sample Images (Optional)

In [ ]:
# Test on validation images (sample from val set)
test_images = list((YOLO_DATASET / "val" / "images").glob("*"))[:10]

print(f"Testing on {len(test_images)} sample images...")

for img_path in test_images:
    results = model.predict(str(img_path), save=True, conf=0.25)
    print(f"  ✓ {img_path.name}")

print(f"\nResults saved to: {RUNS_DIR / project_name}")